In [1]:
# # Install MLflow and its dependencies
# !pip install mlflow==2.9.0
# !pip install lz4==4.3.3
# !pip install numpy==1.26.4
# !pip install pandas==1.3.0
# !pip install scikit-learn==1.6.0
# !pip install scipy==1.11.4
# !pip install xgboost==2.1.3

In [2]:
import pandas as pd
from loguru import logger

import mlflow
from mlflow.tracking import MlflowClient

# Configure logging
logger.add("logs/inference.log")

# Set MLflow tracking URI
mlflow.set_tracking_uri("http://localhost:5001")


def load_latest_model_and_mappings(model_name: str):
    """Load the latest model version and its category mappings"""
    client = MlflowClient()

    try:
        # Get latest model version using aliases instead of stages
        logger.info(f"Attempting to load latest version of model: {model_name}")
        latest_version = client.get_model_version_by_alias(model_name, "current")
        run_id = latest_version.run_id
        logger.info(f"Found model version: {latest_version.version} with run_id: {run_id}")

        if not run_id:
            logger.error("No run_id found for the model version")
            return None, None

        # Load the model
        model = mlflow.pyfunc.load_model(f"models:/{model_name}@current")
        logger.info("Model loaded successfully")

        # Get category mappings from the same run
        try:
            # List artifacts to debug
            logger.info(f"Listing artifacts for run_id: {run_id}")
            artifacts = client.list_artifacts(run_id)
            logger.info(f"Available artifacts: {[art.path for art in artifacts]}")
            
            # Try to load the mappings
            category_mappings = mlflow.artifacts.load_dict(
                f"runs:/{run_id}/category_mappings.json"
            )
            logger.info("Category mappings loaded successfully")
            logger.debug(f"Mappings content: {category_mappings}")
            
            return model, category_mappings

        except Exception as e:
            logger.error(f"Error loading category mappings: {e}")
            return None, None

    except Exception as e:
        logger.error(f"Error loading model: {e}")
        return None, None

In [3]:
# Model name
model_name = "purchase_prediction_model"

# Load model and mappings
model, category_mappings = load_latest_model_and_mappings(model_name)

2024-12-24 23:27:25.156 | INFO     | __main__:load_latest_model_and_mappings:20 - Attempting to load latest version of model: purchase_prediction_model
2024-12-24 23:27:25.180 | INFO     | __main__:load_latest_model_and_mappings:23 - Found model version: 6 with run_id: 58c3a14211e74bb3acad6f6d7443163a
2024/12/24 23:27:25 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.9.0, required: mlflow==2.19.0)
 - xgboost (current: 2.1.3, required: xgboost==2.0.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024-12-24 23:27:25.797 | INFO     | __main__:load_latest_model_and_mappings:31 - Model loaded successfully
2024-12-24 23:27:25.798 | INFO     | __main__:load_latest_model_and_mappings:36 - Listing artifacts for run_id: 58c3a14211e74bb3acad6f6d7443163a
2024-12-24 23:27:25

In [4]:
# Prepare inference data
data = [
    {
        "brand": "sumsung",
        "price": 130.76,
        "event_weekday": 2,
        "category_code_level1": "electronics",
        "category_code_level2": "smartphone",
        "activity_count": 1,
    },
    {
        "brand": "video",
        "price": 130.76,
        "event_weekday": 2,
        "category_code_level1": "electronics",
        "category_code_level2": "smartphone",
        "activity_count": 1,
    },
]

# Convert to DataFrame
df = pd.DataFrame(data)
logger.info(f"Input data shape: {df.shape}")

2024-12-24 23:27:25.841 | INFO     | __main__:<module>:23 - Input data shape: (2, 6)


In [5]:
# Encode categorical columns using saved mappings
for col in ["brand", "category_code_level1", "category_code_level2"]:
    mapping = category_mappings[col]
    # Map values using the saved mapping, with -1 for unseen categories
    df[col] = df[col].map(mapping).fillna(-1)
    logger.info(f"Encoded column {col}")

2024-12-24 23:27:25.847 | INFO     | __main__:<module>:6 - Encoded column brand
2024-12-24 23:27:25.848 | INFO     | __main__:<module>:6 - Encoded column category_code_level1
2024-12-24 23:27:25.849 | INFO     | __main__:<module>:6 - Encoded column category_code_level2


In [6]:
# Make predictions
predictions = model.predict(df)
logger.info(f"Predictions: {predictions}")

2024-12-24 23:27:25.890 | INFO     | __main__:<module>:3 - Predictions: [0.5839172 0.5839172]


In [7]:
df

,brand,price,event_weekday,category_code_level1,category_code_level2,activity_count
0,-1.0,130.76,2,3,12,1
1,-1.0,130.76,2,3,12,1
